In [1]:
import os
import json
import PIL.Image
import random
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from src.data.ImageNet100ClassesDataset import ImageNet100ClassesDataset, prepare_dataloaders_ImageNet100ClassesDataset
from src.attacks.attacks import FastGradientSign, ProjectedGradientDescent
from src.training.Trainer import Trainer
from src.optim.scheduler import CustomScheduler


def seed_everything(seed_value=4995):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def load_model(num_classes = 100, model_path = None, to_cuda = True):
    if not model_path:
        model = torchvision.models.resnet18(pretrained = True)
        input_feat = model.fc.in_features
        
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_state_dict = False
    
    else:
        print("Loaded", model_path)
        model = torchvision.models.resnet18()
        input_feat = model.fc.in_features
        model.fc = nn.Linear(input_feat, num_classes)
        loaded_model = torch.load(model_path)
        model.load_state_dict(loaded_model['model_state_dict'])
        loaded_state_dict = True
        
    if to_cuda:
        model = model.to('cuda')
        
    return model, loaded_state_dict



In [5]:
seed_everything()
root_dir = 'Data/imagenet100classes'

config = {
    'lr': 0.001,
    'batch_size': 64,
    'weight_decay': 0.01
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
resnet, loaded_state_dict = load_model(model_path = 'models/resnet_100_imagenet_adv_training.pt')

for name, param in resnet.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False


train_loader, val_loader = prepare_dataloaders_ImageNet100ClassesDataset(root_dir, batch_size = config['batch_size'])

dataloaders = {
    'train': train_loader,
    'val': val_loader
}

loss_fn = nn.CrossEntropyLoss()

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = config['lr'], weight_decay = config['weight_decay'])
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,5], gamma=0.1)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

classes = list(train_loader.dataset.class_name_dict.values())


pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 20, epsilon = 0.25, return_logits=False)


trainer = Trainer(resnet, loss_fn, classes, training_params, DEVICE, num_epochs = 1, model_name = 'resnet_100_imagenet_adv_training', save_model = True, model_dir = 'models', adversarial_training = True, adversarial_attack = pgd)


trainer.train()
"""

# Fine-tune all layers
resnet, loaded_state_dict = load_model(model_path = 'models/resnet_100_imagenet_fine_tuned.pt')

for name, param in resnet.named_parameters():
    param.requires_grad = True

train_loader, val_loader = prepare_dataloaders_ImageNet100ClassesDataset(root_dir, batch_size = config['batch_size'])
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

loss_fn = nn.CrossEntropyLoss()

train_params = [p for p in resnet.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(train_params, lr = 0.0001, weight_decay = config['weight_decay'])
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,5], gamma=0.1)
scheduler = CustomScheduler(optimizer)

training_params = {
    'dataloaders': dataloaders,
    'optimizer': optimizer,
    'scheduler': scheduler
}

classes = list(train_loader.dataset.class_name_dict.values())
pgd = ProjectedGradientDescent(resnet, loss_fn, iterations = 5, alpha = 2/255, epsilon = 2/255, return_logits=False)

trainer_fine_tune = Trainer(resnet, loss_fn, classes, training_params, DEVICE, num_epochs = 10, model_name = 'resnet_100_imagenet_adv_training_larger_lr_new_scheduler', save_model = True, model_dir = 'models', adversarial_training = True, adversarial_attack = pgd)



Loaded models/resnet_100_imagenet_fine_tuned.pt


In [6]:
trainer_fine_tune.train()


print("Losses:",trainer_fine_tune.epoch_loss)
print("Accuracies:",trainer_fine_tune.epoch_acc)

Current Epoch: 0
Train Loop:
Batch: 0 of 2032. Loss: 4.268786907196045. Mean so far: 4.268786907196045. Mean of 100: 4.268786907196045
Batch: 100 of 2032. Loss: 1.6217896938323975. Mean so far: 2.0763730695932217. Mean of 100: 2.0544489312171934
Batch: 200 of 2032. Loss: 1.5981557369232178. Mean so far: 1.8352575574941303. Mean of 100: 1.591730890274048
Batch: 300 of 2032. Loss: 1.1546604633331299. Mean so far: 1.7283950085655795. Mean of 100: 1.5136012852191925
Batch: 400 of 2032. Loss: 1.2918837070465088. Mean so far: 1.655310793113233. Mean of 100: 1.4353273046016692
Batch: 500 of 2032. Loss: 1.4109734296798706. Mean so far: 1.6087420529234195. Mean of 100: 1.422001404762268
Batch: 600 of 2032. Loss: 1.1374326944351196. Mean so far: 1.5781948932395402. Mean of 100: 1.4251536232233049
Batch: 700 of 2032. Loss: 1.183595895767212. Mean so far: 1.5544046712839994. Mean of 100: 1.4114254373311996
Batch: 800 of 2032. Loss: 1.370367407798767. Mean so far: 1.5354912302616086. Mean of 100: 1

AttributeError: 'CustomScheduler' object has no attribute 'state_dict'

In [ ]:
continue_dict = {
            "best_measure": 0.5624,
            "train_measure_at_best": 0.5940538461538462,
            "train_loss_at_best":  1.448255855883435,
            "best_eval_loss": 1.6207028069073641,
            "best_epoch": 0,
            "latest_epoch": 0,
}